<a href="https://colab.research.google.com/github/bryonbaker/SIT720/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SIT720 - Assignment 1

Bryon Baker
Student number: 85031775

# Import required modules and libraries

In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install matplotlib

You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Function Declarations

## QUESTION 1
* Load the data from supplied data file. 
* Select the first 500 observations/ samples from the dataset. 
* Print the data dimension.


In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass1/hypothyroid.csv'
fullht_df = pd.read_csv(url)
fullht_df.head(n=500)

In [ ]:
# Get the first 500 rows from the dataset and use that for the rest of the assignment.
ht_df = fullht_df.head(n=500)

# Display the dataset's dimension
initialShape = ht_df.shape
print(f"Working dataset dimension is: {initialShape}\n")

## QUESTION 2
* Continue from question 1. 
* Display the number of rows and their indices that have missing data in one or more cells. 
* Now, replace the missing data by the lowest value of the corresponding feature if it is a continuous variable. In case of categorical variable, remove the sample. 
* Print the median values of all features before and after replacing missing data.

In [ ]:
# Find the number of rows that have missing data
ht_df = ht_df.replace('?', np.NaN)              # Replace ? with NaN
numMissing = ht_df.isna().any(axis=1).sum()
print(f"The number of rows with missing values is {numMissing}")

# Define the features that are subject to replacing with the lowest value in the column.
# This series will ne used to define the features that will be updated if they are missing.
continuousFeatures = pd.Series(["age", "TSH","T3","TT4","FTI"], copy = False)

# All features are currently text. Change the data types of the continuous variables to numeric
# so we can perform calculations with them.
ht_df[continuousFeatures] = ht_df[continuousFeatures].apply(pd.to_numeric)

# Calculate the Median prior to replacing missing values
initialMedian = ht_df[continuousFeatures].median(skipna=True)
print("\n=============================")
print("Median Pre-Data Modification:")
print("=============================")
print(f"The Median of each column is:\n{initialMedian}")

# Find the minimum value for each continuous variable
minValues = ht_df.min(axis=0)
print("\n======================================")
print("Updating missing continuous variables:")
print("======================================")
for index, value in continuousFeatures.items():
    print(f"Min value for {value} is {minValues[value]}")
    ht_df[value] = ht_df[value].fillna(minValues[value])

# Calculate the Median value of all continuous features.
# Using the list of continuous features, replace any missing (NaN) values with the lowest value in the column.
# Replace the NaN's of the numeric columns with the mean
ht_df["TSH"] = ht_df["TSH"].fillna(continuousFeatures[1])
# Calculate the new Median value of all continuous features.
# Display the before/after Median values

# Calculate the Median prior to replacing missing values
postMedian = ht_df[continuousFeatures].median(skipna=True)
print("\n===============================")
print("Median Post-Data Modification:")
print("==============================")
print(f"The Median of each column is:\n{postMedian}")

# Now all the continuous variables have had NaN replaced, drop any rows with categorical values of NaN
numMissing = ht_df.isna().any(axis=1).sum()# Recalculate the number of rows left with missing values for assert later.
ht_df = ht_df.dropna()


# Display the dataset's new dimension
print(f"Working dataset dimension is: {ht_df.shape}\n")
print(f"Initial shape: {initialShape[0]} New shape: {ht_df.shape[0]} Number of rows dropped: {numMissing}")

# Check that the new shape is correct based on the number of missing non-continuous values
assert initialShape[0] == ht_df.shape[0]+numMissing

In [ ]:
# Display the new data set for informational purposes
ht_df.head(n=500)

## QUESTION 3

* Continue from question 2.
* Is there any change in data type? If yes, convert them back to appropriate data types. 
* Print all variables with corresponding data type.

In [ ]:
# Change the t/f features to proper boolean values by creating a new column mapped to the original value

boolColumns = pd.Series(["on thyroxine", "pregnant", "I131 treatment", "query hyperthyroid"], copy = False)    # The columns to change types
boolMap = {'t' : True, 'f': False }    # How to map the values
cols = ht_df.columns.tolist()     # Get the order of columns so we can preserve the table structure after we change all the types with row swaps.

for index, value in boolColumns.items():
    oldName = value    # Save the old name for renaming back later
    newName = "new_" + value    # Create a temporary column name
    ht_df[newName] = ht_df[oldName].map(boolMap)    # Adds a new column with proper boolean values
    ht_df = ht_df.drop([oldName], axis=1)    # Drop the old column before renaming the new column to the name just dropped
    ht_df = ht_df.rename(columns={newName : oldName})

ht_df = ht_df[cols]    # Reorder the columns back to what they were before we started.

In [ ]:
print("Updated table structure")
print("=======================")
ht_df.head()

In [ ]:
print("DataFrame variables and corresponding data types:")
print("=================================================")
print(ht_df.dtypes)

## QUESTION 4
* Continue from question 3. 
* Print the total numbers and ratio of male and female patients who are receiving I131 treatment.

In [ ]:
male_i131 = ht_df[(ht_df["I131 treatment"] == True) & (ht_df["sex"] == "M")]
female_i131 = ht_df[(ht_df["I131 treatment"] == True) & (ht_df["sex"] == "F")]
total_i131 = ht_df[ht_df["I131 treatment"] == True]
numMales = ht_df[ht_df["sex"] == "M"].shape[0]
numFemales = ht_df[ht_df["sex"] == "F"].shape[0]

numMalesWithI131 = male_i131.shape[0]
numFemalesWithI131 = female_i131.shape[0]
totalWithI131 = total_i131.shape[0]

print(f"Number of Males receiving I131 treatment is: {numMalesWithI131}")
print(f"Number of Females receiving I131 treatment is: {numFemalesWithI131}")
print(f"Total number of people receiving I131 treatment is: {totalWithI131}")
print(f"Ratio of Males receiving/not receiving I131 Treatment is: {numMalesWithI131/numMales}")
print(f"Ratio of Females receiving/not receiving I131 Treatment is: {numFemalesWithI131/numFemales}")

## QUESTION 5
* Continue from question 3. 
* Print the average TSH and T3 with the patient with Hypothyroid problem. 
* Please report the pattern found in the result, if any.

In [ ]:
# Looking for records with binaryClass == P. Calculate the average for the TSH and T3

averages = ht_df[ht_df["binaryClass"] == "P"][["TSH","T3"]].mean()
print("TSH/T3 Breakdown of Patients at Risk of Hypothyroid Disease:")
print("\tAverage TSH: {:3f} mU/L".format(averages['TSH']))
print("\tAverage T3: {:3f} ng/dl".format(averages['T3']))

### ANALYSIS
When considered in isolation, these results indicate that there is a relationship between the average TSH and T3 values. (TSH: 1.582 and T3: 1.585)
However, this alone should not be used to draw any conclusions because, as shown below, the ranges (0..27.0 vs. 0..5.5) and standard deviations (2.86 vs. 1.08) are very different, indicating that the similar values are coincidence.

In [ ]:
anal = ht_df[ht_df["binaryClass"] == "P"][["TSH","T3"]]
print(f"TSH Breakdown:\n{anal['TSH'].describe()}\n")
print(f"T3 Breakdown:\n{anal['T3'].describe()}")

## QUESTION 6
* Continue from question 3. 
* Print the average TSH and T3 with the patient with Hyperthyroid problem. 
* Please report the pattern found in the result, if any.

In [ ]:
means = ht_df[ht_df["query hyperthyroid"] == True][["TSH","T3"]].mean()
tshMean = means["TSH"] if not pd.isna(means["TSH"]) else 0
t3Mean = means["T3"] if not pd.isna(means["T3"]) else 0

print("TSH/T3 Breakdown of Patients at Risk of Hyperthyroid Disease:")
print("\tAverage TSH: {:.3f} mU/L".format(tshMean))
print("\tAverage T3: {:.3f} ng/dl".format(t3Mean))

## ANALYSIS
...

## QUESTION 7
* Continue from question 3.
* Print the percentage of pregnant patients at risk of Hypothyroid disease.

In [ ]:
#numPreg = ht_df[(ht_df["binaryClass"] == "P") & (ht_df["pregnant"] == True)].shape[0]
preg_df = ht_df[ht_df["pregnant"] == True]
pregHypo_df = preg_df[preg_df["binaryClass"] == "P"]
print("The percentage of pregnant patients at risk of Hypothyroid disease is {:.2f}%".format((pregHypo_df.shape[0]/preg_df.shape[0])*100))

## QUESTION 8
* Continue from question 3. 
* Calculate and print the average TSH and TT4 for pregnant patient at risk of Hypothyroid disease.

In [ ]:
means = pregHypo_df[["TSH","TT4"]].mean()
tshMean = means["TSH"] if not pd.isna(means["TSH"]) else 0
tt4Mean = means["TT4"] if not pd.isna(means["TT4"]) else 0

print("TSH/TT4 Breakdown of Pregnant Patients at Risk of Hypothyroid Disease:")
print("\tAverage TSH: {:.3f} mU/L".format(tshMean))
print("\tAverage TT4: {:.3f} nmol/L".format(tt4Mean))

## QUESTION 9
* Continue from question 3.
* Calculate and print the average FTI and TT4 for pregnant patient who are receiving I131 treatment and are at risk of Hyperthyroid disease.

In [ ]:
#pregHyperI131_df = preg_df[(preg_df["query hyperthyroid"] == True) & (preg_df["I131 treatment"] == True)]
pregHyperI131_df = ht_df[(ht_df["pregnant"] == True) & (ht_df["query hyperthyroid"] == True) & (ht_df["I131 treatment"] == True)]
means = pregHyperI131_df[["FTI","TT4"]].mean()
ftiMean = means["FTI"] if not pd.isna(means["FTI"]) else 0
tt4Mean = means["TT4"] if not pd.isna(means["TT4"]) else 0
print("FTI/TT4 Breakdown of Pregnant Patients at Risk of Hypothyroid Disease:")
print("\tAverage FTI: {:.3f} ug/dL".format(ftiMean))
print("\tAverage TT4: {:.3f} nmol/L".format(tt4Mean))

## QUESTION 10
* Continue from question 3.
* Create and print a dataframe of the FTI, T3, and TSH levels for diferent age groups (based on 10 years intervals)

In [ ]:
# Find min age and max age
# loop through age range in increments of 10: (0->9, 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99)
# Query the main data frame

# Add an age-range column
# set the age-range value based on a query of (age <= minAge && age < maxAge)



In [ ]:
# Calculate the age range
min = ht_df["age"].min()
min = min - min%10        # Round the age down to the nearest 10
max = ht_df["age"].max()
if max % 10 == 0:
    max += 10             # If the max age is divisible by ten create an extra age bracket as we are using 0-9, 10-19, ... 

# Create an age map of specific age to bracket
ageMap = {}
bracket = 0
for age in range(min, max):
    ageMap[age] = "%d-%d" % (bracket, bracket+9)
    if (age+1) % 10 == 0:
        bracket += 10
    
# Add a new column with the age bracket
bracket_df = ht_df
bracket_df["age bracket"] = bracket_df["age"].map(ageMap)
bracket_df = bracket_df.sort_values(by=["age bracket"])
q10_df = bracket_df[{"age bracket", "FTI", "T3", "TSH"}]
print(q10_df.to_string())

## Question 11
* Continue from question 3. 
* Find the average T3 and T4 level of across gender for each age group (based on 10 years interval). 
* Please explain the results.


In [ ]:
# Calculate the age range
min = ht_df["age"].min()
min = min - min%10        # Round the age down to the nearest 10
max = ht_df["age"].max()
if max % 10 == 0:
    max += 10             # If the max age is divisible by ten create an extra age bracket as we are using 0-9, 10-19, ... 
    
print("=====================================================")
print("Average T3 & TT4 levels across gender by age bracket:")
print("=====================================================")
for age in range(min, max, 10):
    bracket = "%d-%d" % (age,age+9)
    print(f"Age Bracket: {bracket}")
    b = bracket_df[bracket_df["age bracket"] == bracket]
    averages = b[["T3", "TT4"]].mean()
    print("Average T3: {:.3f}".format(averages["T3"]))
    print("Average TT4: {:.3f}\n".format(averages["TT4"]))

### Analysis


## Question 12
* Continue from question 3. 
* Draw two scatter plots of FTI level, one against T4 and another against TSH rate. 
* Draw them in two subplots of the same plot.

In [ ]:
fig, ax = plt.subplots(2, figsize=(10, 6))

ax[0].scatter(x = ht_df['FTI'], y = ht_df['TT4'])
ax[0].set_xlabel("FTI (ug/dL)")
ax[0].set_ylabel("TT4 (nmol/L)")

ax[1].scatter(x = ht_df['FTI'], y = ht_df['TSH'])
ax[1].set_xlabel("FTI (ug/dL)")
ax[1].set_ylabel("TSH (mU/L)")

plt.show()

## Question 13
* Continue from question 3.
* Visualize the number of pregnant for each Hyper and Hypo disease group using bar plot. 
* Explain the graph base on your observation.

In [ ]:
# Collect the data for plotting. h1 = Hyperthyroid, H2 = Hypothyroid
h1p = preg_df[preg_df["query hyperthyroid"] == True]
h1n = preg_df[preg_df["query hyperthyroid"] == False]
h2p = preg_df[preg_df["query hyperthyroid"] == True]
h2n = preg_df[preg_df["query hyperthyroid"] == False]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_xlabel("Pregnant Patients by Disease Groups")
ax.set_ylabel("Number in group")

diseaseGroup = ['Hyper Positive', 'Hyper Negative', 'Hypo Positive', 'Hypo Negative']
totals = [h1p.shape[0],h1n.shape[0],h2p.shape[0],h2n.shape[0]]
ax.bar(diseaseGroup,totals)
plt.show()

### Analysis

## Question 14
* Continue from question 3. 
* Draw a group bar diagram of T3, T4, TSH and total number of hypo patients, based on age groups. 
* Explain your observation from the graph.

In [ ]:
# Calculate the age range
min = ht_df["age"].min()
min = min - min%10        # Round the age down to the nearest 10
max = ht_df["age"].max()
if max % 10 == 0:
    max += 10             # If the max age is divisible by ten create an extra age bracket as we are using 0-9, 10-19, ... 

# Define the arrays to hold the data for plotting
labels = []
t3 = []
tt4 = []
tsh = []
hypo_count = []

for age in range(min, max, 10):
    bracket = "%d-%d" % (age,age+9)
    labels.append(bracket)
    sub_bracket_df = bracket_df[bracket_df["age bracket"] == bracket]
    a = sub_bracket_df[["T3", "TT4", "TSH"]].mean()
    t3.append(round(a["T3"],3))
    tt4.append(round(a["TT4"],3))
    tsh.append(round(a["TSH"],3))
    c = sub_bracket_df[sub_bracket_df["binaryClass"]=="P"]
    hypo_count.append(c.shape[0])

print("Raw data for graphing:")
print(f"Age brackets: {labels}")
print(f"Average T3: {t3}")
print(f"Average TT4: {tt4}")
print(f"Average TSH: {t3}")
print(f"Hypothyroid Disease Positive Count: {hypo_count}")


In [ ]:
x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x, t3, width, label='T3 avg')
rects2 = ax.bar(x+(width), tt4, width, label='TT4 avg')
rects3 = ax.bar(x+(width*2), tsh, width, label='TSH avg')
rects4 = ax.bar(x+(width*3), hypo_count, width, label='# Hypothyroid +ve')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_title('Age Bracket')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Print the value on top of each bar
ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)
ax.bar_label(rects3, padding=3)
ax.bar_label(rects4, padding=3)

fig.set_size_inches(25,10)
fig.tight_layout()

plt.show()

### Analysis

## Question 15
* Continue from question 3. 
* Calculate the number of male and female patients affected by hyper disease in each group.

In [ ]:
males = ht_df[(ht_df["sex"] == "M") & (ht_df["query hyperthyroid"] == True)]
females = ht_df[(ht_df["sex"] == "F") & (ht_df["query hyperthyroid"] == True)]
print(f"The number of Males affected by Hyperthyroid disease is {males.shape[0]}")
print(f"The number of Females affected by Hyperthyroid disease is {females.shape[0]}")